In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import skimage.io as skio
from skimage.transform import rescale

In [2]:
def Loading_image(address):
  img_2D_matrix = skio.imread(address, plugin="tifffile")
  return img_2D_matrix

In [3]:
#spliting data to 3 chanels:

def split(data):
  h, w = data.shape
  treshold  = int(np.floor(h / 3.0))
  image = np.zeros((treshold, w, 3), dtype=np.uint16)
  image[:, :, 0] = data[:treshold]
  image[:, :, 1] = data[treshold:2*treshold]
  image[:, :, 2] = data[2*treshold:3*treshold]

  return image

In [4]:
# Cropping 10% off the borders of the image
def crop(img):
	
	height, width = img.shape
	new_height, new_width = int(height/1.11), int(width/1.11)
	return img[height-new_height:new_height, width-new_width:new_width]

In [5]:
def L2_penalty(ch1, ch2_moved):
  Penalty = (((ch1-ch2_moved)**2).sum()).sum()

  return Penalty

In [6]:
def displacement(ch1, ch2, org):
  #searching box:

  lower_I = org[0] - 10
  higher_I = org[0] + 10
  lower_J = org[1] - 10
  higher_J = org[1] + 10

  Penalty_min = 1e20
  disp_vector = [0, 0]
  for i in range(lower_I, higher_I):
    for j in range(lower_J, higher_J):
      ch2_moved = np.roll(np.roll(ch2, i, axis = 0), j, axis = 1)
      Penalty = L2_penalty(ch1, ch2_moved)
      if(Penalty < Penalty_min):
        Penalty_min = Penalty
        disp_vector = [i, j]

  return disp_vector

In [7]:
def pyramid(ch1, ch2, tre,itr):
  h, w = ch1.shape
  if(max(h, w) > tre):
    aa = cv2.pyrDown(ch1)
    bb = cv2.pyrDown(ch2)
    ch1_lev = rescale(ch1, 0.5)
    ch2_lev = rescale(ch2, 0.5)
    itr += 1
    recursive_disp_vector = pyramid(ch1_lev, ch2_lev, tre,itr)
    scaled_org = [x*2 for x in recursive_disp_vector] 
    disp_vector = displacement(ch1, ch2, scaled_org)
    
    #print(itr,'  ',disp_vector)
  else:
    disp_vector = displacement(ch1, ch2, [10, 10])
    #print(itr,'  ',disp_vector)

  return disp_vector


In [8]:
#calculating the displacement vector for each channel and move it:
def coloring(image):
  tre = 40
  h, w = image[:, :, 0].shape
  colored_img = np.zeros((h, w, 3), dtype=np.uint16)

  ch1 = image[:, :, 0]
  ch2 = image[:, :, 1]
  ch3 = image[:, :, 2]

  disp_vector2 = pyramid(ch1, ch2, tre,0)
  print('second channel displacement vector is:' , disp_vector2)
  ch2_moved_opt = np.roll(np.roll(ch2, disp_vector2[0], axis = 0), disp_vector2[1], axis = 1)

  disp_vector3 = pyramid(ch1, ch3, tre, 0)
  print('third channel displacement vector is:', disp_vector3)
  ch3_moved_opt = np.roll(np.roll(ch3, disp_vector3[0], axis = 0), disp_vector3[1], axis = 1)

  colored_img[:, :, 0] = ch1
  colored_img[:, :, 1] = ch2_moved_opt
  colored_img[:, :, 2] = ch3_moved_opt

  return colored_img

In [ ]:
#Girls :

In [9]:
from skimage import *
from skimage.util import img_as_ubyte

imd_2D_matrix = Loading_image('/content/master-pnp-prok-01000-01043a.tif')
image = split(imd_2D_matrix)
image1 = image
#image1 = img_as_ubyte(image)
#plt.imshow(image1)

In [10]:
temp = crop(image1[:, :, 0])
h, w = temp.shape
image_croped = np.zeros((h, w, 3), dtype=np.uint16)
image_croped[:, :, 0] = temp
image_croped[:, :, 1] = crop(image1[:, :, 1])
image_croped[:, :, 2] = crop(image1[:, :, 2])

#displacement calculation:
colored_img = coloring(image_croped)

#croping the margines:
temp0 = crop(colored_img[:, :, 0])
h, w = temp0.shape
res_croped = np.zeros((h, w, 3), dtype=np.uint16)
res_croped[:, :, 0] = temp0
res_croped[:, :, 1] = crop(colored_img[:, :, 1])
res_croped[:, :, 2] = crop(colored_img[:, :, 2])


second channel displacement vector is: [-10, 13]
third channel displacement vector is: [6, 19]


In [11]:
from google.colab.patches import cv2_imshow

In [12]:
!curl -o logo.png https://colab.research.google.com/img/colab_favicon_256px.png
import cv2
im = img_as_ubyte(res_croped)
#plt.imshow(im)
cv2_imshow(im)
cv2.imwrite('Girls-result-07.jpg', im)

Output hidden; open in https://colab.research.google.com to view.

In [13]:
!curl -o logo.png https://colab.research.google.com/img/colab_favicon_256px.png
import cv2
im = img_as_ubyte(image1)
#plt.imshow(im)
cv2_imshow(im)
cv2.imwrite('Girls-org.jpg', im)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
#Train

In [14]:
from skimage import *
from skimage.util import img_as_ubyte

imd_2D_matrix = Loading_image('/content/master-pnp-prok-00400-00402a.tif')
image = split(imd_2D_matrix)
image1 = image
#image1 = img_as_ubyte(image)
#plt.imshow(image1)

In [15]:
temp = crop(image1[:, :, 0])
h, w = temp.shape
image_croped = np.zeros((h, w, 3), dtype=np.uint16)
image_croped[:, :, 0] = temp
image_croped[:, :, 1] = crop(image1[:, :, 1])
image_croped[:, :, 2] = crop(image1[:, :, 2])

#displacement calculation:
colored_img = coloring(image_croped)

#croping the margines:
temp0 = crop(colored_img[:, :, 0])
h, w = temp0.shape
res_croped = np.zeros((h, w, 3), dtype=np.uint16)
res_croped[:, :, 0] = temp0
res_croped[:, :, 1] = crop(colored_img[:, :, 1])
res_croped[:, :, 2] = crop(colored_img[:, :, 2])

second channel displacement vector is: [72, 20]
third channel displacement vector is: [142, 29]


In [19]:
!curl -o logo.png https://colab.research.google.com/img/colab_favicon_256px.png
import cv2
im = img_as_ubyte(colored_img)
#plt.imshow(im)
cv2_imshow(im)
cv2.imwrite('Museum-result-06.jpg', im)

Output hidden; open in https://colab.research.google.com to view.

In [17]:
!curl -o logo.png https://colab.research.google.com/img/colab_favicon_256px.png
import cv2
im = img_as_ubyte(image1)
#plt.imshow(im)
cv2_imshow(im)
cv2.imwrite('Museum-org.jpg', im)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
#Museum

In [20]:
from skimage import *
from skimage.util import img_as_ubyte

imd_2D_matrix = Loading_image('/content/master-pnp-prok-00400-00458a.tif')
image = split(imd_2D_matrix)
image1 = image
#image1 = img_as_ubyte(image)
#plt.imshow(image1)

In [21]:
temp = crop(image1[:, :, 0])
h, w = temp.shape
image_croped = np.zeros((h, w, 3), dtype=np.uint16)
image_croped[:, :, 0] = temp
image_croped[:, :, 1] = crop(image1[:, :, 1])
image_croped[:, :, 2] = crop(image1[:, :, 2])

#displacement calculation:
colored_img = coloring(image_croped)

#croping the margines:
temp0 = crop(colored_img[:, :, 0])
h, w = temp0.shape
res_croped = np.zeros((h, w, 3), dtype=np.uint16)
res_croped[:, :, 0] = temp0
res_croped[:, :, 1] = crop(colored_img[:, :, 1])
res_croped[:, :, 2] = crop(colored_img[:, :, 2])

second channel displacement vector is: [37, 2]
third channel displacement vector is: [97, 22]


In [25]:
!curl -o logo.png https://colab.research.google.com/img/colab_favicon_256px.png
import cv2
im = img_as_ubyte(res_croped)
#plt.imshow(im)
cv2_imshow(im)
cv2.imwrite('Train-result-05.jpg', im)

Output hidden; open in https://colab.research.google.com to view.

In [23]:
!curl -o logo.png https://colab.research.google.com/img/colab_favicon_256px.png
import cv2
im = img_as_ubyte(image1)
#plt.imshow(im)
cv2_imshow(im)
cv2.imwrite('Train-org.jpg', im)

Output hidden; open in https://colab.research.google.com to view.